In [1]:
from script_fire import fire_csv

In [2]:
df_final = fire_csv()

Data fire shape: (29723, 7)
Data fire only code metro: (26021, 7)
Data fire with INSEE code: (25831, 16)
Data fire with department area: (25831, 17)
Agg data fire: (1668, 6)
Agg data fire + two columns: (1668, 8)
Data fire + agg data fire: (25831, 19)


In [3]:
print('fire score department by quarter:', df_final.shape)

fire score department by quarter: (1668, 3)


In [ ]:
df_final.to_csv('data_risk_fire.csv', index=False)

In [4]:
df_final.head()

,department,year_quarter,fire_score
824,41,2014T1,0.0
598,33,2014T1,0.0
1133,63,2014T1,0.0
1610,88,2014T1,0.0
516,30,2014T1,0.0


In [5]:
import geopandas as gpd

gdf = gpd.read_file(
    "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements.geojson"
)

print(gdf.columns)

Index(['code', 'nom', 'geometry'], dtype='object')


In [9]:
gdf_merged = gdf.merge(df_final, 
                       left_on='code', 
                       right_on='department', 
                       how='inner') \
                .drop(columns='department')
print(gdf_merged.shape)
gdf_merged.head()

(1668, 5)


,code,nom,geometry,year_quarter,fire_score
0,2A,Corse-du-Sud,"MULTIPOLYGON (((8.5864 41.87389, 8.58667 41.87...",2014T1,0.0
1,2A,Corse-du-Sud,"MULTIPOLYGON (((8.5864 41.87389, 8.58667 41.87...",2014T2,1.0
2,2A,Corse-du-Sud,"MULTIPOLYGON (((8.5864 41.87389, 8.58667 41.87...",2014T3,1.0
3,2A,Corse-du-Sud,"MULTIPOLYGON (((8.5864 41.87389, 8.58667 41.87...",2014T4,1.0
4,2A,Corse-du-Sud,"MULTIPOLYGON (((8.5864 41.87389, 8.58667 41.87...",2015T1,0.0


In [ ]:
fig = px.choropleth(
    gdf_merged,
    geojson=gdf,
    locations='code',            # colonne dans ton DataFrame
    featureidkey='properties.code',    # chemin dans le GeoJSON
    color='fire_score',
    animation_frame='year_quarter',    # variable temporelle
    color_continuous_scale='Reds',
    range_color=(0, 3),                # 0 = Low, 3 = Critical
    labels={'fire_score': 'Fire Score'},
    title='Fire Risk Score by Department Over Time'
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
